Based on https://colab.research.google.com/drive/1jocViLorbwWIkTXKwxCOV9HLTaDDgCaw?usp=sharing

In [1]:
#@title
!pip install transformers==3.1.0

     |████████████████████████████████| 890kB 5.3MB/s 
     |████████████████████████████████| 3.0MB 12.1MB/s 
     |████████████████████████████████| 1.1MB 31.1MB/s 
     |████████████████████████████████| 890kB 38.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b71d64993e6c7c0416206329547dc97fbd47bec252c7f8c4b6a8d6abda00be51
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import pipeline
import pandas as pd
import csv
from google.colab import files

In [3]:
classifier = pipeline("zero-shot-classification")
#classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

#classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3")
#classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", device=0) # to utilize GPU

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.

In [4]:
# load data
filename = 'https://s3-eu-west-1.amazonaws.com/files.innerdoc.com/christmas-emotion-detection/songs.csv'
separator = ';'
encoding = 'utf-8'
df = pd.read_csv(filename, sep=separator, header=0, encoding=encoding)
df

,Id,Artist,Title,Album,Released,Youtube_url,Lyrics
0,1,Mariah Carey,All I want for Christmas is you,Merry Christmas,1994,https://www.youtube.com/watch?v=yXQViqx6GMY,I don't want a lot for Christmas\nThere is jus...
1,2,Wham!,Last Christmas,Now – The Christmas Album,1985,https://www.youtube.com/watch?v=E8gmARGvPlI,Last Christmas I gave you my heart\nBut the ve...
2,3,Band Aid,Do they Know it's Christmas,Do They Know It's Christmas?,1984,https://www.youtube.com/watch?v=bjQzJAKxTrE,"It's Christmas time, there's no need to be afr..."
3,4,The Pogues,Fairytale Of New York,If I Should Fall from Grace with God,1988,https://www.youtube.com/watch?v=j9jbdgZidu8,It was Christmas Eve babe\nIn the drunk tank\n...
4,5,Michael Bublé,Santa Claus Is Coming To Town,Christmas,2011,https://www.youtube.com/watch?v=R8CBoVc_OMI,You better watch out\nYou better not cry\nYou ...
5,6,Chris Rea,Driving Home For Christmas,On the Beach,1986,https://www.youtube.com/watch?v=DDt3u2Ev1cI,"I'm driving home for Christmas\nOh, I can't wa..."
6,7,Shakin' Stevens,Merry Christmas Everyone,Merry Christmas Everyone,1985,https://www.youtube.com/watch?v=N-PyWfVkjZc,Snow is falling\nAll around me\nChildren playi...
7,8,The Jackson 5,I Saw Mommy Kissing Santa Claus,Jackson 5 Christmas Album,1970,https://www.youtube.com/watch?v=O89sPooBhyE,Wow! Mommy's kissing Santa Claus!\nI saw Mommy...
8,9,"John Lennon, Yoko Ono",Happy Xmas (War Is Over),Working Class Hero: The Definitive Lennon,1971,https://www.youtube.com/watch?v=flA5ndOyZbI,So this is Christmas\nAnd what have you done\n...
9,10,The Pretenders,2000 Miles,Learning to Crawl,1983,https://www.youtube.com/watch?v=AEyGZlBdkaA,"She's gone 2000 miles, it's very far\nThe snow..."


In [12]:
# load data
filename = 'https://s3-eu-west-1.amazonaws.com/files.innerdoc.com/christmas-emotion-detection/plutchik.csv'
separator = ';'
encoding = 'utf-8'
df_emotions = pd.read_csv(filename, sep=separator, header=0, encoding=encoding)
df_emotions['emotion_label'] = df_emotions['primary_emotion'] + ' ' + df_emotions['intense_emotion'] #df_emotions[['primary_emotion', 'intense_emotion']].apply(lambda x: ' '.join(x), axis=1)
candidate_labels =  df_emotions[df_emotions['id'] > -1].emotion_label.values.tolist()
print(candidate_labels)
df_emotions
#[["terror fear", "protection"], ["amazement surprise", "orientation"], ["grief sadness", "reintegration"], ["loathing disgust", "rejection"], ["rage anger", "destruction"], ["vigilance anticipation", "exploration"], ["ecstasy joy", "reproduction"], ["admiration trust", "affiliation"]]

['fear terror', 'surprise amazement', 'sadness grief', 'disgust loathing', 'anger rage', 'anticipation vigilance', 'joy ecstasy', 'trust admiration']


,id,intense_emotion,primary_emotion,mild_emotion,combination_1,combination_1_result,combination_2,combination_2_result,opposite_primary_emotion,stimulus_event,cognitive_appraisal,behavior,function,effect,intense_emotion_color,primary_emotion_color,mild_emotion_color,tekst_color,emotion_label
0,1,terror,fear,apprehension,surprise,awe,trust,submission,anger,threat,danger,escape,"running away,find safety",protection,#a3cc3f,#b4d465,#d9e8b1,#85a82d,fear terror
1,2,amazement,surprise,distraction,sadness,disappointment,fear,awe,anticipation,unexpected event,what is it?,"stopping, alerting",gain time to orient,orientation,#61c082,#85ca9b,#c1e1c9,#3a8055,surprise amazement
2,3,grief,sadness,pensiveness,disgust,remorse,surprise,disappointment,joy,loss of valued object,"abandonment, isolation",crying for help,reattach to lost object,reintegration,#40bded,#71c9f1,#b9e2f8,#3d87a4,sadness grief
3,4,loathing,disgust,boredom,anger,contempt,sadness,remorse,trust,gruesome object,poison,"vomiting, pushing away",eject poison,rejection,#964097,#a665a9,#cca9d1,#a549a8,disgust loathing
4,5,rage,anger,annoyance,anticipation,aggressiveness,disgust,contempt,fear,obstacle,enemy,attacking,destroy obstacle,destruction,#dd0c81,#e35995,#efaac7,#e35995,anger rage
5,6,vigilance,anticipation,interest,joy,optimism,anger,aggressiveness,surprise,new territory,what's out there?,"examining, mapping",knowledge of territory,exploration,#faa719,#fcb84d,#ffd8a4,#ee7c28,anticipation vigilance
6,7,ecstasy,joy,serenity,trust,love,anticipation,optimism,sadness,gain of valued object,posses,"retaining, courting",gain resources,reproduction,#f4eb24,#f8f06b,#f9f39b,#e7c436,joy ecstasy
7,8,admiration,trust,acceptance,fear,submission,joy,love,disgust,group member,friend,"grooming, sharing",mutual support,affiliation,#dddd26,#dae25b,#eaefad,#abbc17,trust admiration
8,-1,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,#d3d3d3,#d3d3d3,#d3d3d3,#d3d3d3,unknown unknown


In [13]:
def classify(lyrics):
  sequence = lyrics.split('\n')
  hypothesis_template = "The sentiment of this text is {}."

  resp = classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template, multi_class=True)
  if type(resp) != list:
    resp = [resp]

  return resp


def parse_lyrics(song_id, lyric):
  sentences = lyric.split('\n')
  results = []
  for sent_id, sent in enumerate(sentences):
    print(f'\n{sent}')
    if len(sent) > 12:
      response = classify(sent)
    else:
      response = []

    for r in response:
      labels = []
      for label, score in zip(r['labels'], r['scores']):
        if score > .85 or score >= max(r['scores']):
          label_id = df_emotions[df_emotions['emotion_label'] == label]['id'].values[0]
          labels.append([label_id, label, score])
          print(f'    {label} ({round(score, 2)})')
      results.append([song_id, sent_id, sent, labels])

    if not response:
      results.append([song_id, sent_id, sent, [[-1, 'UNK', 0.0]] ])

  return results


In [14]:
# parse all lyrics
total = []
for index, row in df.iterrows():
  if index > 99:
    break
  print(f'\n{row.Artist} - {row.Title} ({row.Released})\n')
  results = parse_lyrics(row.Id, row.Lyrics)
  for r in results:
    total.append(r)

# export  
filename = 'songlabels.csv'
with open(filename, 'w', newline='\n') as f:
  wr = csv.writer(f, delimiter=';', quoting=csv.QUOTE_ALL)
  wr.writerow(['song_id', 'sent_id', 'sent', 'labels'])
  for row in total:
    wr.writerow(row)

files.download(filename)

print('\n\n\nDATA EXPORTED!')


Mariah Carey - All I want for Christmas is you (1994)


I don't want a lot for Christmas
    anticipation vigilance (0.3)

There is just one thing I need
    anticipation vigilance (0.64)

I don't care about the presents
    disgust loathing (0.96)

Underneath the Christmas tree
    surprise amazement (0.84)

I just want you for my own
    trust admiration (0.96)

More than you could ever know
    surprise amazement (0.99)

Make my wish come true
    anticipation vigilance (0.52)

All I want for Christmas is you, yeah
    trust admiration (0.96)

I don't want a lot for Christmas
    anticipation vigilance (0.3)

There is just one thing I need
    anticipation vigilance (0.64)

And I don't care about the presents
    disgust loathing (0.88)

Underneath the Christmas tree
    surprise amazement (0.84)

I don't need to hang my stocking
    surprise amazement (0.8)

There upon the fireplace
    surprise amazement (0.74)

Santa Claus won't make me happy
    disgust loathing (0.98)
    sadn

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>




DATA EXPORTED!


To do multi-class classification, simply pass `multi_class=True`. In this case, the scores will be independent, but each will fall between 0 and 1.

Here's an example of sentiment classification: 

So how does this method work?

The underlying model is trained on the task of Natural Language Inference (NLI), which takes in two sequences and determines whether they contradict each other, entail each other, or neither.

This can be adapted to the task of zero-shot classification by treating the sequence which we want to classify as one NLI sequence (called the premise) and turning a candidate label into the other (the hypothesis). If the model predicts that the constructed premise _entails_ the hypothesis, then we can take that as a prediction that the label applies to the text. Check out [this blog post](https://joeddav.github.io/blog/2020/05/29/ZSL.html) for a more detailed explanation.

By default, the pipeline turns labels into hypotheses with the template `This example is {class_name}.`. This works well in many settings, but you can also customize this for your specific setting. Let's add another review to our above sentiment classification example that's a bit more challenging:

The second example is a bit harder. Let's see if we can improve the results by using a hypothesis template which is more specific to the setting of review sentiment analysis. Instead of the default, `This example is {}.`, we'll use, `The sentiment of this review is {}.` (where `{}` is replaced with the candidate class name)

The model is fine-tuned on XNLI which includes 15 languages: Arabic, Bulgarian, Chinese, English, French, German, Greek, Hindi, Russian, Spanish, Swahili, Thai, Turkish, Urdu, and Vietnamese. The base model is trained on 85 more, so the model will work to some degree for any of those in the XLM RoBERTa training corpus (see the full list in appendix A of the [XLM Roberata paper](https://arxiv.org/abs/1911.02116)).

See the [model page](https://huggingface.co/joeddav/xlm-roberta-large-xnli) for more.